In [ ]:
!pip install networkx matplotlib pandas seaborn numpy

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from collections import Counter
import random

# Core Functions: Graph Creation & Deadlock Detection

In [ ]:
def g(p,r,a,q):
    """Create Resource Allocation Graph"""
    G = nx.DiGraph()
    G.add_nodes_from(p)
    G.add_nodes_from(r)
    for x,y in a.items():
        if y: G.add_edge(x,y)
    for x,y in q.items():
        if y: G.add_edge(x,y)
    return G

def d(G):
    """Detect deadlock by finding cycles"""
    try:
        return True, nx.find_cycle(G)
    except:
        return False, None

def s(G,c=None,t="RAG"):
    """Visualize Resource Allocation Graph"""
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(7,5))
    nx.draw_networkx_nodes(G,pos,node_size=3000,node_color="lightblue")
    nx.draw_networkx_labels(G,pos)
    if c:
        nx.draw_networkx_edges(G,pos,edgelist=[e for e in G.edges() if e not in c],arrowstyle='-|>',arrowsize=20,edge_color='blue')
        nx.draw_networkx_edges(G,pos,edgelist=c,arrowstyle='-|>',arrowsize=20,edge_color='red',width=2)
    else:
        nx.draw_networkx_edges(G,pos,edgelist=G.edges(),arrowstyle='-|>',arrowsize=20,edge_color='blue')
    plt.title(t)
    plt.axis('off')
    plt.show()

# Original Functions: Single & Multi Scenario

In [ ]:
def run_single():
    """Interactive single scenario"""
    n=int(input("processes: "))
    m=int(input("resources: "))
    p=[f"P{i+1}" for i in range(n)]
    r=[f"R{i+1}" for i in range(m)]
    a={}
    for x in r:
        y=int(input(f"{x} alloc to (1-{n},0): "))
        a[x]=None if y==0 else p[y-1]
    q={}
    for x in p:
        y=int(input(f"{x} request (1-{m},0): "))
        q[x]=None if y==0 else r[y-1]
    G=g(p,r,a,q)
    f,c=d(G)
    s(G,c,"Interactive RAG")
    print("DEADLOCK!" if f else "SAFE")

def run_multi():
    """Run 5 predefined scenarios"""
    cases=[
        ({"R1":"P1","R2":"P2"},{"P1":"R2","P2":"R1"}),
        ({"R1":"P1","R2":None},{"P1":"R2","P2":None}),
        ({"R1":None,"R2":None},{"P1":None,"P2":None}),
        ({"R1":"P2","R2":None},{"P1":"R1","P2":None}),
        ({"R1":"P2","R2":"P1"},{"P1":"R2","P2":"R1"})
    ]
    res=[]
    for i,(a,q) in enumerate(cases):
        p=["P1","P2"]
        r=["R1","R2"]
        G=g(p,r,a,q)
        f,c=d(G)
        res.append(0 if f else 1)
        s(G,c,f"Scenario {i+1} RAG")
    plt.figure(figsize=(7,5))
    plt.bar([f"S{i+1}" for i in range(len(res))],res,color=['green' if x==1 else 'red' for x in res])
    plt.ylabel("State (1=Safe,0=Deadlock)")
    plt.title("EDA: Safe vs Deadlock")
    plt.show()

# Enhanced EDA: Helper Functions

In [ ]:
def generate_random_scenario(n_processes, n_resources):
    """Generate a random resource allocation scenario"""
    p = [f"P{i+1}" for i in range(n_processes)]
    r = [f"R{i+1}" for i in range(n_resources)]
    
    # Random allocations (some resources may be unallocated)
    a = {}
    for res in r:
        if random.random() > 0.3:  # 70% chance of allocation
            a[res] = random.choice(p)
        else:
            a[res] = None
    
    # Random requests (some processes may not request)
    q = {}
    for proc in p:
        if random.random() > 0.3:  # 70% chance of request
            q[proc] = random.choice(r)
        else:
            q[proc] = None
    
    return p, r, a, q

def calculate_graph_metrics(G):
    """Calculate various graph metrics"""
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    
    metrics = {
        'nodes': n_nodes,
        'edges': n_edges,
        'edge_density': n_edges / (n_nodes * (n_nodes - 1)) if n_nodes > 1 else 0,
        'avg_degree': sum(dict(G.degree()).values()) / n_nodes if n_nodes > 0 else 0,
        'max_degree': max(dict(G.degree()).values()) if n_nodes > 0 else 0,
    }
    
    return metrics

# Monte Carlo Simulation

In [ ]:
def monte_carlo_simulation(n_simulations=500, min_proc=2, max_proc=8, min_res=2, max_res=8):
    """Run Monte Carlo simulation with random scenarios"""
    print(f"\n🔄 Running Monte Carlo simulation with {n_simulations} scenarios...\n")
    
    results = []
    
    for i in range(n_simulations):
        n_proc = random.randint(min_proc, max_proc)
        n_res = random.randint(min_res, max_res)
        
        p, r, a, q = generate_random_scenario(n_proc, n_res)
        G = g(p, r, a, q)
        has_deadlock, cycle = d(G)
        
        metrics = calculate_graph_metrics(G)
        
        results.append({
            'scenario': i+1,
            'processes': n_proc,
            'resources': n_res,
            'p_r_ratio': n_proc / n_res,
            'edges': metrics['edges'],
            'edge_density': metrics['edge_density'],
            'avg_degree': metrics['avg_degree'],
            'max_degree': metrics['max_degree'],
            'cycle_length': len(cycle) if cycle else 0,
            'deadlock': 1 if has_deadlock else 0
        })
        
        if (i + 1) % 100 == 0:
            print(f"✓ Completed {i+1}/{n_simulations} scenarios")
    
    print(f"\n✅ Simulation complete!\n")
    return pd.DataFrame(results)

# Comprehensive EDA Analysis

In [ ]:
def run_comprehensive_eda():
    """Run comprehensive EDA with multiple analyses"""
    
    # 1. Monte Carlo Simulation
    df = monte_carlo_simulation(n_simulations=500)
    
    # Basic Statistics
    print("="*60)
    print("📊 SUMMARY STATISTICS")
    print("="*60)
    deadlock_count = df['deadlock'].sum()
    safe_count = len(df) - deadlock_count
    print(f"Total Scenarios: {len(df)}")
    print(f"Deadlocks: {deadlock_count} ({deadlock_count/len(df)*100:.1f}%)")
    print(f"Safe: {safe_count} ({safe_count/len(df)*100:.1f}%)")
    print(f"\nAverage Processes: {df['processes'].mean():.1f}")
    print(f"Average Resources: {df['resources'].mean():.1f}")
    print(f"Average Edge Density: {df['edge_density'].mean():.3f}")
    print()
    
    # 2. Create comprehensive visualizations
    fig = plt.figure(figsize=(16, 12))
    
    # Plot 1: Overall Distribution
    ax1 = plt.subplot(3, 3, 1)
    safe_dead = df['deadlock'].value_counts()
    colors = ['green', 'red']
    plt.bar(['Safe', 'Deadlock'], [safe_dead.get(0, 0), safe_dead.get(1, 0)], color=colors)
    plt.title('Overall State Distribution', fontweight='bold')
    plt.ylabel('Count')
    
    # Plot 2: Deadlock Rate by Process Count
    ax2 = plt.subplot(3, 3, 2)
    proc_deadlock = df.groupby('processes')['deadlock'].agg(['mean', 'count'])
    proc_deadlock = proc_deadlock[proc_deadlock['count'] >= 10]  # Filter low samples
    plt.plot(proc_deadlock.index, proc_deadlock['mean'] * 100, marker='o', linewidth=2, markersize=8)
    plt.title('Deadlock Rate by Process Count', fontweight='bold')
    plt.xlabel('Number of Processes')
    plt.ylabel('Deadlock Rate (%)')
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Deadlock Rate by Resource Count
    ax3 = plt.subplot(3, 3, 3)
    res_deadlock = df.groupby('resources')['deadlock'].agg(['mean', 'count'])
    res_deadlock = res_deadlock[res_deadlock['count'] >= 10]
    plt.plot(res_deadlock.index, res_deadlock['mean'] * 100, marker='s', linewidth=2, markersize=8, color='orange')
    plt.title('Deadlock Rate by Resource Count', fontweight='bold')
    plt.xlabel('Number of Resources')
    plt.ylabel('Deadlock Rate (%)')
    plt.grid(True, alpha=0.3)
    
    # Plot 4: P/R Ratio vs Deadlock
    ax4 = plt.subplot(3, 3, 4)
    df['ratio_bin'] = pd.cut(df['p_r_ratio'], bins=10)
    ratio_deadlock = df.groupby('ratio_bin')['deadlock'].mean() * 100
    ratio_deadlock.plot(kind='bar', color='purple', alpha=0.7)
    plt.title('Deadlock Rate by P/R Ratio', fontweight='bold')
    plt.xlabel('Process/Resource Ratio')
    plt.ylabel('Deadlock Rate (%)')
    plt.xticks(rotation=45, ha='right')
    
    # Plot 5: Edge Density vs Deadlock
    ax5 = plt.subplot(3, 3, 5)
    plt.scatter(df['edge_density'], df['deadlock'], alpha=0.3, s=30)
    density_deadlock = df.groupby(pd.cut(df['edge_density'], bins=10))['deadlock'].mean()
    density_centers = [interval.mid for interval in density_deadlock.index]
    plt.plot(density_centers, density_deadlock.values, color='red', linewidth=2, marker='o')
    plt.title('Edge Density vs Deadlock', fontweight='bold')
    plt.xlabel('Edge Density')
    plt.ylabel('Deadlock (0=Safe, 1=Deadlock)')
    plt.grid(True, alpha=0.3)
    
    # Plot 6: Correlation Heatmap
    ax6 = plt.subplot(3, 3, 6)
    corr_cols = ['processes', 'resources', 'p_r_ratio', 'edge_density', 'avg_degree', 'deadlock']
    corr_matrix = df[corr_cols].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix', fontweight='bold')
    
    # Plot 7: Cycle Length Distribution (for deadlocks only)
    ax7 = plt.subplot(3, 3, 7)
    deadlock_df = df[df['deadlock'] == 1]
    if len(deadlock_df) > 0:
        cycle_counts = deadlock_df['cycle_length'].value_counts().sort_index()
        plt.bar(cycle_counts.index, cycle_counts.values, color='crimson', alpha=0.7)
        plt.title('Cycle Length Distribution (Deadlocks)', fontweight='bold')
        plt.xlabel('Cycle Length')
        plt.ylabel('Frequency')
    else:
        plt.text(0.5, 0.5, 'No Deadlocks Found', ha='center', va='center', fontsize=12)
        plt.title('Cycle Length Distribution')
    
    # Plot 8: Processes vs Resources Heatmap
    ax8 = plt.subplot(3, 3, 8)
    heatmap_data = df.groupby(['processes', 'resources'])['deadlock'].mean().unstack(fill_value=0)
    sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='RdYlGn_r', center=0.5,
                cbar_kws={'label': 'Deadlock Rate'})
    plt.title('Deadlock Rate Heatmap (P×R)', fontweight='bold')
    plt.xlabel('Resources')
    plt.ylabel('Processes')
    
    # Plot 9: Avg Degree vs Deadlock
    ax9 = plt.subplot(3, 3, 9)
    degree_bins = pd.cut(df['avg_degree'], bins=10)
    degree_deadlock = df.groupby(degree_bins)['deadlock'].mean() * 100
    degree_deadlock.plot(kind='line', marker='d', linewidth=2, markersize=8, color='teal')
    plt.title('Deadlock Rate by Avg Node Degree', fontweight='bold')
    plt.xlabel('Average Node Degree')
    plt.ylabel('Deadlock Rate (%)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('deadlock_eda_comprehensive.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # 3. Statistical Insights
    print("="*60)
    print("🔍 KEY INSIGHTS")
    print("="*60)
    
    # Correlation analysis
    print("\n1. CORRELATION WITH DEADLOCK:")
    correlations = df[['processes', 'resources', 'p_r_ratio', 'edge_density', 'avg_degree']].corrwith(df['deadlock'])
    for feature, corr in correlations.items():
        print(f"   {feature}: {corr:+.3f}")
    
    # Optimal configurations
    print("\n2. SAFEST CONFIGURATIONS:")
    safe_df = df[df['deadlock'] == 0]
    print(f"   Avg Processes: {safe_df['processes'].mean():.1f}")
    print(f"   Avg Resources: {safe_df['resources'].mean():.1f}")
    print(f"   Avg P/R Ratio: {safe_df['p_r_ratio'].mean():.2f}")
    print(f"   Avg Edge Density: {safe_df['edge_density'].mean():.3f}")
    
    # High-risk configurations
    print("\n3. HIGHEST RISK CONFIGURATIONS:")
    deadlock_df = df[df['deadlock'] == 1]
    if len(deadlock_df) > 0:
        print(f"   Avg Processes: {deadlock_df['processes'].mean():.1f}")
        print(f"   Avg Resources: {deadlock_df['resources'].mean():.1f}")
        print(f"   Avg P/R Ratio: {deadlock_df['p_r_ratio'].mean():.2f}")
        print(f"   Avg Edge Density: {deadlock_df['edge_density'].mean():.3f}")
    
    # Edge density threshold
    print("\n4. EDGE DENSITY ANALYSIS:")
    low_density = df[df['edge_density'] < 0.3]['deadlock'].mean()
    high_density = df[df['edge_density'] >= 0.3]['deadlock'].mean()
    print(f"   Low density (<0.3): {low_density*100:.1f}% deadlock rate")
    print(f"   High density (≥0.3): {high_density*100:.1f}% deadlock rate")
    
    print("\n" + "="*60)
    print("💡 RECOMMENDATIONS:")
    print("="*60)
    print("1. Maintain more resources than processes (R > P)")
    print("2. Keep edge density below 0.3 for safer operation")
    print("3. Monitor systems with P/R ratio > 1.0 closely")
    print("4. Implement resource ordering to break potential cycles")
    print("="*60 + "\n")
    
    return df

# Main Execution: Choose Your Mode

In [ ]:
print("\n" + "="*60)
print("  DEADLOCK DETECTION & EDA SYSTEM")
print("="*60)
print("\nOptions:")
print("1 - Single Interactive Scenario")
print("2 - Multi-Scenario Demo (5 predefined cases)")
print("3 - Comprehensive EDA (Monte Carlo + Analysis)")
print("="*60)

c = int(input("\nSelect option (1/2/3): "))

if c == 1:
    run_single()
elif c == 2:
    run_multi()
elif c == 3:
    df_results = run_comprehensive_eda()
    
    # Optional: Save results to CSV
    save = input("\nSave results to CSV? (y/n): ").lower()
    if save == 'y':
        df_results.to_csv('deadlock_simulation_results.csv', index=False)
        print("Results saved to 'deadlock_simulation_results.csv'")
else:
    print("Invalid option!")